## Advent of Code 2024 - Day 14

In [86]:
from rich import print
from httpx import request
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from itertools import product
import math
import functools

%load_ext rich

The rich extension is already loaded. To reload it, use:
  %reload_ext rich


In [2]:
def parse_input(path):
    # Read file and split into lines
    with open(path, "r") as file:
        result = file.read().split("\n")
    # Optional: Remove any empty lines if needed
    return [line.split(" ") for line in result]

In [3]:
sample_input = parse_input("sample.txt")
actual_input = parse_input("input.txt")

## Part 1

In [4]:
sample_input



[
    ['p=0,4', 'v=3,-3'],
    ['p=6,3', 'v=-1,-3'],
    ['p=10,3', 'v=-1,2'],
    ['p=2,0', 'v=2,-1'],
    ['p=0,0', 'v=1,3'],
    ['p=3,0', 'v=-2,-2'],
    ['p=7,6', 'v=-1,-3'],
    ['p=3,0', 'v=-1,-2'],
    ['p=9,3', 'v=2,3'],
    ['p=7,3', 'v=-1,2'],
    ['p=2,4', 'v=2,-3'],
    ['p=9,5', 'v=-3,-3']
]

In [19]:
def parse_p_v(line):
    return (
        tuple(map(int, line[0].split("=")[1].split(","))),
        tuple(map(int, line[1].split("=")[1].split(","))),
    )

In [27]:
class Robot:
    def __init__(self, p, v, MAX_X, MAX_Y):
        self.pos = p
        self.velocity = v
        self.step = 0

        self.MAX_X = MAX_X
        self.MAX_Y = MAX_Y

    def take_step(self):
        new_p = tuple(map(sum, zip(self.pos, self.velocity)))
        new_x, new_y = new_p

        if new_x < 0:
            new_x = self.MAX_X - abs(new_x)
        elif new_x >= self.MAX_X:
            new_x = new_x - self.MAX_X

        if new_y < 0:
            new_y = self.MAX_Y - abs(new_y)
        elif new_y >= self.MAX_Y:
            new_y = new_y - self.MAX_Y

        self.pos = (new_x, new_y)
        self.step += 1

    def take_steps(self, n_steps=10):
        for _ in range(n_steps):
            self.take_step()

    def __repr__(self):
        return f"Robot at {self.pos} at timestep {self.step}"

In [93]:
class Floor:
    def __init__(self, MAX_X, MAX_Y):
        self.robots = []
        self.MAX_X = MAX_X
        self.MAX_Y = MAX_Y

    def add_robot(self, robot):
        self.robots.append(robot)

    def check_grid(self):
        self.robots_on_floor = np.zeros((self.MAX_Y, self.MAX_X))

        for robot in self.robots:
            x, y = robot.pos
            self.robots_on_floor[y][x] += 1

        return self.robots_on_floor

    def draw_floor(self):
        self.floor = np.zeros((self.MAX_Y, self.MAX_X), dtype=str)
        for y in range(self.MAX_Y):
            for x in range(self.MAX_X):
                if (x, y) in [robot.pos for robot in self.robots]:
                    self.floor[y][x] = "#"
                else:
                    self.floor[y][x] = "."

        # return self.floor
        return np.where(self.floor == "#", 1, 0)


In [94]:
def solution_1(input, MAX_X, MAX_Y):
    robots = [Robot(*parse_p_v(line), MAX_X, MAX_Y) for line in input]
    floor = Floor(MAX_X, MAX_Y)
    for robot in robots:
        floor.add_robot(robot)

    for robot in robots:
        robot.take_steps(100)

    floor_grid = floor.check_grid()

    q1 = [floor_grid[i][: MAX_X // 2] for i in range(MAX_Y // 2)]
    q2 = [floor_grid[i][(MAX_X // 2) + 1 :] for i in range(MAX_Y // 2)]
    q3 = [floor_grid[i][: MAX_X // 2] for i in range((MAX_Y // 2) + 1, MAX_Y)]
    q4 = [floor_grid[i][(MAX_X // 2) + 1 :] for i in range((MAX_Y // 2) + 1, MAX_Y)]

    return np.sum(q1) * np.sum(q2) * np.sum(q3) * np.sum(q4)


In [95]:
print(f"Part 1 - Sample: {solution_1(sample_input, MAX_X=11, MAX_Y=7)}")
print(f"Part 1 - Actual: {solution_1(actual_input, MAX_X=101, MAX_Y=103)}")


Part 1 - Sample: 12.0

Part 1 - Actual: 230435667.0

## Part 2

In [96]:
sample_input


[
    ['p=0,4', 'v=3,-3'],
    ['p=6,3', 'v=-1,-3'],
    ['p=10,3', 'v=-1,2'],
    ['p=2,0', 'v=2,-1'],
    ['p=0,0', 'v=1,3'],
    ['p=3,0', 'v=-2,-2'],
    ['p=7,6', 'v=-1,-3'],
    ['p=3,0', 'v=-1,-2'],
    ['p=9,3', 'v=2,3'],
    ['p=7,3', 'v=-1,2'],
    ['p=2,4', 'v=2,-3'],
    ['p=9,5', 'v=-3,-3']
]

In [115]:
import torch as t

floor_images = []


def solution_2(input, MAX_X, MAX_Y):
    robots = [Robot(*parse_p_v(line), MAX_X, MAX_Y) for line in input]
    floor = Floor(MAX_X, MAX_Y)
    for robot in robots:
        floor.add_robot(robot)

    i = 0

    while True:
        for robot in robots:
            robot.take_step()

        i += 1
        floor_grid = floor.check_grid()

        floor_images.append(t.Tensor(floor_grid))

        if np.max(floor_grid) == 1:
            print(f"Found at step {i}")
            return floor

In [116]:
final_floor = solution_2(actual_input, MAX_X=101, MAX_Y=103)

Found at step 7709